# Simulation experiment 

Run entire simulation experiment multiple times to generate confidence interval

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../")
from functions import pipelines

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [2]:
# Output files
local_dir = "/home/alexandra/Documents/"

similarity_uncorrected_file = os.path.join(
    local_dir,
    "Data",
    "Batch_effects",
    "output",
    "analysis_0_similarity_uncorrected.pickle")

ci_uncorrected_file = os.path.join(
    local_dir,
    "Data",
    "Batch_effects",
    "output",
    "analysis_0_ci_uncorrected.pickle")

In [3]:
# Run multiple simulations - uncorrected
iterations = range(10) 
num_cores = 5
results = Parallel(n_jobs=num_cores, verbose=100)(delayed(pipelines.simple_simulation_experiment_uncorrected)() for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed:  3.6min
[Parallel(n_jobs=5)]: Done   2 out of  10 | elapsed:  3.6min remaining: 14.3min
[Parallel(n_jobs=5)]: Done   3 out of  10 | elapsed:  3.6min remaining:  8.3min
[Parallel(n_jobs=5)]: Done   4 out of  10 | elapsed:  3.6min remaining:  5.4min
[Parallel(n_jobs=5)]: Done   5 out of  10 | elapsed:  3.6min remaining:  3.6min
[Parallel(n_jobs=5)]: Done   6 out of  10 | elapsed:  7.0min remaining:  4.7min
[Parallel(n_jobs=5)]: Done   7 out of  10 | elapsed:  7.1min remaining:  3.0min
[Parallel(n_jobs=5)]: Done   8 out of  10 | elapsed:  7.1min remaining:  1.8min
[Parallel(n_jobs=5)]: Done  10 out of  10 | elapsed:  7.1min remaining:    0.0s
[Parallel(n_jobs=5)]: Done  10 out of  10 | elapsed:  7.1min finished


In [4]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score,score,score,score,score,score
number of experiments,,,,,,,,,,
1,0.999994,0.999994,0.999994,0.999994,0.999994,0.999994,0.999994,0.999994,0.999994,0.999994
2,0.900114,0.901288,0.902440,0.900818,0.900778,0.904073,0.902896,0.901961,0.900216,0.903033
5,0.608636,0.605191,0.606569,0.609099,0.608854,0.611319,0.607703,0.606072,0.611641,0.610445
10,0.132619,0.133192,0.130262,0.129535,0.132469,0.128529,0.129495,0.129523,0.128140,0.132855
20,0.046617,0.044475,0.043572,0.046758,0.043869,0.045561,0.042914,0.038106,0.042140,0.042367
50,0.144240,0.137678,0.155857,0.142285,0.154535,0.143254,0.141079,0.153755,0.144659,0.149630
100,0.238890,0.238895,0.240395,0.242342,0.240542,0.239128,0.241622,0.241735,0.238874,0.244530
500,0.728082,0.738931,0.748747,0.713077,0.733708,0.718701,0.718507,0.717600,0.739200,0.713642
1000,0.936237,0.934146,0.938236,0.940443,0.937659,0.929441,0.937888,0.939195,0.931769,0.937928


In [1]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

NameError: name 'all_svcca_scores' is not defined

In [ ]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

In [ ]:
# Get confidence interval for each row (number of experiments)
# z-score for 95% confidence interval 
err = std_scores*1.96

In [ ]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

In [ ]:
mean_scores

In [ ]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_uncorrected_file)
ci.to_pickle(ci_uncorrected_file)